# Sprint ディープラーニングフレームワーク1

In [1]:
import warnings
warnings.filterwarnings('ignore')

- condaの環境をjupyter上で切り替えられるようにした [参考](https://qiita.com/yoppe/items/38005f415a5b8b884c7d)

In [2]:
import tensorflow as tf
tf.__version__

'1.14.0'

[参考](https://arakan-pgm-ai.hatenablog.com/entry/2018/06/21/090000)

## 1.このSprintについて

### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

### どのように学ぶか
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

## 2.コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。

tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

### 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）

- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

- パラメーターを更新していく
- パラメーターに基づいて特徴量を変換する
- 正解データとの誤差を取り、パラメーターに反映させる

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードでは`Iris.csv`が同じ階層にある想定です。

[Iris Species](https://www.kaggle.com/uciml/iris/data)

目的変数は`Species`ですが、3種類ある中から以下の2種類のみを取り出して使用します。

- Iris-versicolor
- Iris-virginica

### 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

《サンプルコード》

＊バージョン1.5から1.14の間で動作を確認済みです。

In [3]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [4]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="/Users/ikeda/Desktop/dive/diveintocode-ml/Downlowd_data/datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        # total_loss /= n_samples
        total_loss /= total_batch
        # total_acc /= n_samples
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, total_loss : 160.5252, val_loss : 4.7656, total_acc : 0.479, val_acc : 0.500
Epoch 1, total_loss : 56.5071, val_loss : 1.4596, total_acc : 0.521, val_acc : 0.688
Epoch 2, total_loss : 38.9204, val_loss : 32.5276, total_acc : 0.550, val_acc : 0.375
Epoch 3, total_loss : 8.6474, val_loss : 1.1030, total_acc : 0.600, val_acc : 0.812
Epoch 4, total_loss : 7.4924, val_loss : 4.1855, total_acc : 0.657, val_acc : 0.750
Epoch 5, total_loss : 8.7862, val_loss : 1.9870, total_acc : 0.571, val_acc : 0.750
Epoch 6, total_loss : 1.8896, val_loss : 0.2040, total_acc : 0.814, val_acc : 0.938
Epoch 7, total_loss : 0.8564, val_loss : 0.6131, total_acc : 0.929, val_acc : 0.938
Epoch 8, total_loss : 2.0130, val_loss : 3.3206, total_acc : 0.829, val_acc : 0.688
Epoch 9, total_loss : 1.4171, val_loss : 1.1333, total_acc : 0.843, val_acc : 0.938
test_acc : 0.900


- 重みを初期化する必要があった
    - tf.Variableで定義、tf.global_variables_initializer()で初期化、tf.Session.runで初期化実行
- エポックのループが必要だった
    - 同様
- パラメーターを更新していく
    - 直接的な記述はない
    - optimizerで最適化の計算式を定義して、train_opで損失を最小化していくようにoptimizerに命令している
- パラメーターに基づいて特徴量を変換する
    - 全結合は算式を入れ、活性化関数は直接指定している
- 正解データとの誤差を取り、パラメーターに反映させる
    - optimizerで指定して行っている

## 3.他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。

- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットの`train.csv`の中で、目的変数`Species`に含まれる3種類全てを分類できるモデルを作成してください。

[Iris Species](https://www.kaggle.com/uciml/iris/data)

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

**《ヒント》**

以下の2箇所は2クラス分類特有の処理です。

```
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
```

```
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
```

メソッドは以下のように公式ドキュメントを確認してください。

[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)

[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)

＊tf.signとtf.math.signは同じ働きをします。

In [5]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="/Users/ikeda/Desktop/dive/diveintocode-ml/Downlowd_data/datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
# df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
# one_hotへ変換
y = y.astype(np.int)
y = np.identity(3)[y]
# y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
# n_classes = 1
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)    
# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
softmax_out = tf.nn.softmax(logits, axis=1)
correct_pred = tf.equal(tf.argmax(Y), tf.argmax(softmax_out))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        # total_loss /= n_samples
        total_loss /= total_batch
        # total_acc /= n_samples
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, total_loss : 14.8077, val_loss : 8.3558, total_acc : 0.467, val_acc : 0.333
Epoch 1, total_loss : 2.3085, val_loss : 5.9788, total_acc : 0.667, val_acc : 0.333
Epoch 2, total_loss : 1.7133, val_loss : 4.4943, total_acc : 0.767, val_acc : 0.333
Epoch 3, total_loss : 2.0106, val_loss : 2.1327, total_acc : 0.767, val_acc : 1.000
Epoch 4, total_loss : 0.8129, val_loss : 4.6772, total_acc : 0.867, val_acc : 0.667
Epoch 5, total_loss : 0.2064, val_loss : 2.2840, total_acc : 0.833, val_acc : 0.667
Epoch 6, total_loss : 0.5030, val_loss : 0.8661, total_acc : 0.867, val_acc : 1.000
Epoch 7, total_loss : 0.6853, val_loss : 2.2376, total_acc : 0.867, val_acc : 1.000
Epoch 8, total_loss : 0.4493, val_loss : 4.5558, total_acc : 0.833, val_acc : 1.000
Epoch 9, total_loss : 0.4182, val_loss : 1.0328, tot

### 【問題4】House Pricesのモデルを作成

回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中の`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使ってください。説明変数はさらに増やしても構いません。

分類問題と回帰問題の違いを考慮してください。

In [6]:
"""
TensorFlowで実装したニューラルネットワークを使いHousePriceデータセットを回帰予測する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="/Users/ikeda/Desktop/dive/diveintocode-ml/Downlowd_data/train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
objective_variable = 'SalePrice'
features = ['GrLivArea', 'YearBuilt']

y = df[objective_variable]
X = df.loc[:, features]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Xを標準化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_val = std.transform(X_val)
X_test = std.transform(X_test)
# Xを対数変換
# X_train = np.log(X_train)
# X_val = np.log(X_val)
# X_test = np.log(X_test)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    # layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)    
# 目的関数

# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.keras.losses.MSE(y_true=Y, y_pred=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# softmax_out = tf.nn.softmax(logits, axis=1)
# correct_pred = tf.equal(tf.argmax(Y), tf.argmax(softmax_out))
# 指標値計算
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    tf.set_random_seed(0)
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        # total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            # loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            # total_acc += acc
        # total_loss /= n_samples
        total_loss /= total_batch
        # total_acc /= n_samples
        # total_acc /= total_batch
        # val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        # print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
        print("Epoch {}, total_loss : {:,.1f}, val_loss : {:,.1f}".format(epoch, total_loss, val_loss))
    # test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    # print("test_acc : {:.3f}".format(test_acc))
    print("test_loss : {:,}".format(test_loss))

Epoch 0, total_loss : 37,477,081,066.2, val_loss : 30,491,168,768.0
Epoch 1, total_loss : 18,866,371,649.4, val_loss : 4,367,513,088.0
Epoch 2, total_loss : 2,967,428,497.0, val_loss : 1,545,556,608.0
Epoch 3, total_loss : 2,123,104,573.6, val_loss : 1,543,588,736.0
Epoch 4, total_loss : 2,120,721,137.0, val_loss : 1,543,033,344.0
Epoch 5, total_loss : 2,121,772,094.0, val_loss : 1,542,090,624.0
Epoch 6, total_loss : 2,122,393,634.4, val_loss : 1,541,294,336.0
Epoch 7, total_loss : 2,122,845,174.5, val_loss : 1,540,660,608.0
Epoch 8, total_loss : 2,123,213,965.6, val_loss : 1,540,154,752.0
Epoch 9, total_loss : 2,123,527,546.2, val_loss : 1,539,745,152.0
test_loss : 2,925,100,544.0


### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

In [7]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを多値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# one_hotへ変換
y_test = np.identity(10)[y_test]
y_train = np.identity(10)[y_train]

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 20
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)    
# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
softmax_out = tf.nn.softmax(logits, axis=1)
correct_pred = tf.equal(tf.argmax(Y), tf.argmax(softmax_out))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        # total_loss /= n_samples
        total_loss /= total_batch
        # total_acc /= n_samples
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, total_loss : 18.2086, val_loss : 1.2814, total_acc : 0.726, val_acc : 0.300
Epoch 1, total_loss : 0.7806, val_loss : 0.6746, total_acc : 0.520, val_acc : 0.300
Epoch 2, total_loss : 0.4998, val_loss : 0.7915, total_acc : 0.558, val_acc : 0.400
Epoch 3, total_loss : 0.4674, val_loss : 0.5932, total_acc : 0.571, val_acc : 0.500
Epoch 4, total_loss : 0.4968, val_loss : 0.5952, total_acc : 0.592, val_acc : 0.300
Epoch 5, total_loss : 0.4524, val_loss : 0.6163, total_acc : 0.611, val_acc : 0.500
Epoch 6, total_loss : 0.4251, val_loss : 0.6503, total_acc : 0.636, val_acc : 0.500
Epoch 7, total_loss : 0.3913, val_loss : 0.5652, total_acc : 0.650, val_acc : 0.400
Epoch 8, total_loss : 0.3583, val_loss : 0.6322, total_acc : 0.659, val_acc : 0.500
Epoch 9, total_loss : 0.3565, val_loss : 0.6999, total_acc : 0.687, val_acc : 0.700
Epoch 10, total_loss : 0.3831, val_loss : 0.6193, total_acc : 0.691, val_acc : 0.700
Epoch 11, total_loss : 0.3913, val_loss : 0.7379, total_acc : 0.694, val_a